The goal of this notebook is to compare two populations...one sampled from the NHANES dataset directly and one sampled from the Gaussian distributions which were in turn obtained from the NHANES dataset...hence the goal is to validate the Gaussian distributions we obtained...

First we read the df with the person information as obtained from the Gaussian distributions, then we create Person-objects, and then the population object. Finally we compare this population and the NHANESDirectSamplePopulation in baseline characteristics, characteristics over time, and cv events.

In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

from microsim.population import (NHANESDirectSamplePopulation, 
                                 build_people_using_nhanes_for_sampling,
                                 microsimToNhanes, Population, get_nhanes_population)
from microsim.risk_model_repository import RiskModelRepository
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.education import Education
from microsim.treatment import DefaultTreatmentsType,  TreatmentStrategiesType
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.afib_model import AFibPrevalenceModel
from microsim.pvd_model import PVDPrevalenceModel
from microsim.outcome import Outcome, OutcomeType
from microsim.person import Person
from microsim.population_model_repository import PopulationModelRepository
from microsim.cohort_risk_model_repository import (CohortStaticRiskFactorModelRepository,
                                                   CohortDynamicRiskFactorModelRepository,
                                                   CohortDefaultTreatmentModelRepository)
from microsim.outcome_model_repository import OutcomeModelRepository

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
os.chdir(microsimDir+"/CODE/microsim")

dataDir = microsimDir+"/NOTEBOOKS/DATA"

popSize = 100000

In [2]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [3]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})

In [4]:
df = pd.read_csv(dataDir+"/nhanes-persons-from-Gaussians.csv")
df.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,39,39.332549,32.093633,289.420901,202.594404,6.356258,190.692873,108.778183,1.151077,147.131136,81.747572
1,42688,1,0,1,False,1,0,0,0.0,30,57.942825,27.667434,199.841558,119.231151,5.769845,90.956660,92.577832,0.879097,128.707967,61.416109
2,43025,1,0,1,False,1,0,0,0.0,71,50.824894,23.164605,179.578231,85.613429,5.245120,117.216043,92.101304,0.556529,128.105878,67.220136
3,43390,1,0,1,False,1,0,0,0.0,49,42.460672,22.419467,203.219952,192.760294,5.032002,122.120623,90.043771,0.794121,137.643906,84.633473
4,44501,1,0,1,False,1,0,0,0.0,26,57.765745,27.510509,241.259803,137.484893,5.194220,136.485802,88.475559,0.977545,150.569880,87.679994


In [5]:
df["age"].describe()

count    5448.000000
mean       44.616557
std        16.274878
min        16.000000
25%        32.000000
50%        43.000000
75%        56.000000
max        93.000000
Name: age, dtype: float64

In [6]:
df.shape

(5448, 20)

In [7]:
df = df.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()
df.shape

(5448, 21)

In [8]:
def build_person(x, initializationModelRepository):
    """Takes all Person-instance-related data via x and initializationModelRepository and organizes it,
       passes the organized data to the Person class and returns a Person instance."""
    
    rng = np.random.default_rng()
        
    name = x.name
    
    personStaticRiskFactors = {
                        StaticRiskFactorsType.RACE_ETHNICITY.value: NHANESRaceEthnicity(int(x.raceEthnicity)),
                        StaticRiskFactorsType.EDUCATION.value: Education(int(x.education)),
                        StaticRiskFactorsType.GENDER.value: NHANESGender(int(x.gender)),
                        StaticRiskFactorsType.SMOKING_STATUS.value: SmokingStatus(int(x.smokingStatus))}
        
    #use this to get the bounds imposed on the risk factors in a bit
    rfRepository = RiskModelRepository()
        
    #TO DO: find a way to include everything here, including the rfs that need initialization
    #the PVD model would be easy to implement, eg with an estimate_next_risk_for_patient_characteristics function
    #but the AFIB model would be more difficult because it relies on statsmodel_logistic_risk file
    #for now include None, in order to create the risk factor lists correctly at the Person instance
    personDynamicRiskFactors = dict()
    for rfd in DynamicRiskFactorsType:
        #if rfd==DynamicRiskFactorsType.ALCOHOL_PER_WEEK:
        #    personDynamicRiskFactors[rfd.value] = AlcoholCategory.get_category_for_consumption(rfRepository.apply_bounds(rfd.value, x[microsimToNhanes[rfd.value]]))
        #else:
        if (rfd!=DynamicRiskFactorsType.PVD) & (rfd!=DynamicRiskFactorsType.AFIB):
            personDynamicRiskFactors[rfd.value] = rfRepository.apply_bounds(rfd.value, x[rfd.value])
    personDynamicRiskFactors[DynamicRiskFactorsType.AFIB.value] = None
    personDynamicRiskFactors[DynamicRiskFactorsType.PVD.value] = None

    #Q: do we need otherLipid treatment? I am not bringing it to the Person objects for now.
    #A: it is ok to leave it out as we do not have a model to update this. It is also very rarely taking place in the population anyway.
    #also: used to have round(x.statin) but NHANES includes statin=2...
    personDefaultTreatments = {
                        DefaultTreatmentsType.STATIN.value: bool(x.statin),
                        #DefaultTreatmentsType.OTHER_LIPID_LOWERING_MEDICATION_COUNT.value: x.otherLipidLowering,
                        DefaultTreatmentsType.ANTI_HYPERTENSIVE_COUNT.value: x.antiHypertensiveCount}

    personTreatmentStrategies = dict(zip([strategy.value for strategy in TreatmentStrategiesType],
                                         #[None for strategy in range(len(TreatmentStrategiesType))]))
                                         [{"status": None} for strategy in range(len(TreatmentStrategiesType))]))

    personOutcomes = dict(zip([outcome for outcome in OutcomeType],
                                  [list() for outcome in range(len(OutcomeType))]))
    #add pre-simulation stroke outcomes
    #selfReportStrokeAge=x.selfReportStrokeAge
    #Q: we should not add the stroke outcome in case of "else"?
    #if selfReportStrokeAge is not None and selfReportStrokeAge > 1:
    #        selfReportStrokeAge = selfReportStrokeAge if selfReportStrokeAge <= x.age else x.age
    #        personOutcomes[OutcomeType.STROKE].append((selfReportStrokeAge, StrokeOutcome(False, None, None, None, priorToSim=True)))
    #add pre-simulation mi outcomes
    #selfReportMIAge=rng.integers(18, x.age) if x.selfReportMIAge == 99999 else x.selfReportMIAge
    #if selfReportMIAge is not None and selfReportMIAge > 1:
    #        selfReportMIAge = selfReportMIAge if selfReportMIAge <= x.age else x.age
    #        personOutcomes[OutcomeType.MI].append((selfReportMIAge, Outcome(OutcomeType.MI, False, priorToSim=True)))

    person = Person(name,
                   personStaticRiskFactors,
                   personDynamicRiskFactors,
                   personDefaultTreatments,
                   personTreatmentStrategies,
                   personOutcomes)

    #TO DO: find a way to initialize these rfs above with everything else
    person._pvd = [initializationModelRepository[DynamicRiskFactorsType.PVD].estimate_next_risk(person)]
    person._afib = [initializationModelRepository[DynamicRiskFactorsType.AFIB].estimate_next_risk(person)]
    return person


In [9]:
initializationModelRepository = {DynamicRiskFactorsType.AFIB: AFibPrevalenceModel(), 
                                     DynamicRiskFactorsType.PVD: PVDPrevalenceModel()}

In [10]:
dfSample = df.sample(popSize, weights=df.WTINT2YR, replace=True)

In [11]:
people = dfSample.apply(lambda x: build_person(x, initializationModelRepository), axis=1)

In [12]:
#sets the unique identifier for each Person instance
noneList = list(map(lambda person, i: setattr(person, "_index", i), people, range(len(people)))) 

In [13]:
popModelRepository = PopulationModelRepository(CohortDynamicRiskFactorModelRepository(),
                                                           CohortDefaultTreatmentModelRepository(),
                                                           OutcomeModelRepository(),
                                                           CohortStaticRiskFactorModelRepository()) 
pop = Population(people, popModelRepository)

In [14]:
pop._people.shape

(100000,)

In [15]:
#pop2 = get_nhanes_population(1999)
pop2 = NHANESDirectSamplePopulation(popSize, 1999)
pop2.advance(1)
#pop2.print_baseline_summary()

In [16]:
pop.print_baseline_summary_comparison(pop2)

                                                     self                              other
                                                     min    0.25   med    0.75    max   min    0.25   med    0.75    max
                                               age   18.0   33.0   44.6   55.0   93.0   18.0   30.0   43.9   55.0   85.0
                                               sbp   75.0  112.2  123.7  133.5  206.2   72.7  109.3  122.7  131.3  231.3
                                               dbp   38.0   65.3   72.2   78.8  117.0   40.0   65.3   72.5   79.3  132.0
                                               a1c    3.6    5.0    5.4    5.6   10.6    2.5    5.0    5.4    5.5   15.1
                                               hdl   11.3   42.9   52.5   60.8  112.5    8.0   40.0   50.8   59.0  151.0
                                               ldl   41.0  106.8  127.9  147.3  264.3   28.0   98.0  123.0  145.0  354.0
                                              trig   27.0  1

In [17]:
popages = pop.get_age_of_all_years_in_sim()
popageCounts = pop.get_age_counts(popages)
popages2 = pop2.get_age_of_all_years_in_sim()
popageCount2s = pop2.get_age_counts(popages2)

In [46]:
print("age pop-count pop2-count %diff") 
for key in sorted(popageCounts):
    if key in popageCount2s:
        pcDiff =  (popageCounts[key] - popageCount2s[key])/popageCount2s[key]
        print(f"{key} {popageCounts[key]:>6.0f} {popageCount2s[key]:>10.0f} {pcDiff:>9.2f}")

age pop-count pop2-count %diff
18   1343       1707     -0.21
19    891       2102     -0.58
20   1036       1901     -0.46
21   1492       2196     -0.32
22   1486       2376     -0.37
23   1301       1858     -0.30
24   1679       1758     -0.04
25   1190       1654     -0.28
26   2080       2059      0.01
27   1479       1538     -0.04
28   1731       2229     -0.22
29   1970       2206     -0.11
30   2226       2010      0.11
31   2475       1978      0.25
32   2055       2701     -0.24
33   2097       1943      0.08
34   3044       2038      0.49
35   2479       2444      0.01
36   2634       2011      0.31
37   2383       2505     -0.05
38   2028       2172     -0.07
39   2915       2681      0.09
40   2604       2284      0.14
41   2535       2054      0.23
42   2587       2129      0.22
43   1801       1785      0.01
44   2039       2078     -0.02
45   1746       2073     -0.16
46   2327       2164      0.08
47   2186       1781      0.23
48   2064       1611      0.28
49   195

In [19]:
popageCount2s

{18: 1707,
 19: 2102,
 20: 1901,
 21: 2196,
 22: 2376,
 23: 1858,
 24: 1758,
 25: 1654,
 26: 2059,
 27: 1538,
 28: 2229,
 29: 2206,
 30: 2010,
 31: 1978,
 32: 2701,
 33: 1943,
 34: 2038,
 35: 2444,
 36: 2011,
 37: 2505,
 38: 2172,
 39: 2681,
 40: 2284,
 41: 2054,
 42: 2129,
 43: 1785,
 44: 2078,
 45: 2073,
 46: 2164,
 47: 1781,
 48: 1611,
 49: 1612,
 50: 1228,
 51: 1814,
 52: 1536,
 53: 1491,
 54: 1717,
 55: 1135,
 56: 1371,
 57: 1121,
 58: 803,
 59: 1135,
 60: 1472,
 61: 872,
 62: 1106,
 63: 825,
 64: 1039,
 65: 1096,
 66: 985,
 67: 982,
 68: 979,
 69: 794,
 70: 913,
 71: 1029,
 72: 665,
 73: 709,
 74: 586,
 75: 739,
 76: 546,
 77: 717,
 78: 603,
 79: 635,
 80: 619,
 81: 482,
 82: 471,
 83: 323,
 84: 344,
 85: 1480}

In [ ]:
personsOut = list(filter( lambda x: ((x._sbp[0]>=72.) & (x._sbp[0]<=81.)) | ((x._sbp[0]>=207.) & (x._sbp[0]<=266.)) , pop2._people))

In [ ]:
len(personsOut)

In [ ]:
personsOutUnique = set(map( lambda x: x._name, personsOut) )

In [ ]:
len(personsOutUnique)

In [20]:
%%time
pop.advance(17, None, nWorkers=5)

CPU times: user 31.7 s, sys: 3.67 s, total: 35.4 s
Wall time: 7min 26s


In [21]:
%%time
pop2.advance(17, None, nWorkers=5)

CPU times: user 51.2 s, sys: 11.2 s, total: 1min 2s
Wall time: 8min 15s


In [22]:
pop.print_lastyear_summary_comparison(pop2)

                                                     self                              other
                                                     min    0.25   med    0.75    max   min    0.25   med    0.75    max
                                               age   34.0   47.0   57.3   67.0  102.0   35.0   45.0   56.0   65.0  102.0
                                               sbp   80.3  123.0  131.9  139.9  209.4   78.6  120.2  130.1  138.4  262.9
                                               dbp   43.1   74.3   81.9   89.1  129.0   47.4   73.6   81.9   89.9  152.5
                                               a1c    4.2    5.5    5.8    6.1    8.9    4.0    5.5    5.8    6.1   10.6
                                               hdl    5.4   39.4   52.4   64.1  143.1    5.4   35.0   49.8   62.4  169.9
                                               ldl    9.7   95.4  112.1  129.5  209.0    8.1   94.1  112.8  133.1  288.4
                                              trig    9.0  1

In [23]:
%%time
pop.print_cv_standardized_rates()

standardized rates (per 100,000)    all        black   
                            mi      215.4      208.0
                        stroke      146.4      238.0
                         death      801.8      938.3
                            cv      361.8      446.0
                         noncv      751.6      879.2
                      dementia      257.8      501.0
CPU times: user 4min 11s, sys: 25.5 s, total: 4min 36s
Wall time: 4min 41s


In [24]:
%%time
pop2.print_cv_standardized_rates()

standardized rates (per 100,000)    all        black   
                            mi      231.6      205.3
                        stroke      152.6      258.7
                         death      858.5      972.0
                            cv      384.4      464.1
                         noncv      806.3      910.2
                      dementia      274.7      559.7
CPU times: user 4min 13s, sys: 25.5 s, total: 4min 38s
Wall time: 4min 43s


In [ ]:
np.mean(list(map(lambda x: int(x._afib[-1]), pop._people)))

In [ ]:
np.quantile(list(map(lambda x: int(x._afib[-1]), pop._people)), 1.)

In [ ]:
%%time
pop.print_dementia_incidence()

In [ ]:
nYears=5
nSimulations = 4
for bpMedsAdded in [1]:
    miRRList = list()
    strokeRRList = list()
    print(f"\nbpMedsAdded={bpMedsAdded}")
    for i in range(nSimulations):
        treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
        controlPop = NHANESDirectSamplePopulation(popSize, 1999)
        treatmentStrategies = TreatmentStrategyRepository()
        #treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()
        treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddNBPMedsTreatmentStrategy(bpMedsAdded)
        treatmentPop.advance_parallel(1, treatmentStrategies = treatmentStrategies, nWorkers=nWorkers)
        treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
        treatmentPop.advance_parallel(nYears-1, treatmentStrategies = treatmentStrategies, nWorkers=nWorkers)
        controlPop.advance_parallel(nYears, nWorkers=nWorkers)
        controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE),
                                       controlPop._people)))/controlPop._n
        treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE),
                                         treatmentPop._people)))/treatmentPop._n
        strokeRR = treatmentRisk/controlRisk
        controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI),
                                       controlPop._people)))/controlPop._n
        treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI),
                                         treatmentPop._people)))/treatmentPop._n
        miRR = treatmentRisk/controlRisk
        miRRList += [miRR]
        strokeRRList += [strokeRR]
        print(f"\t\tsimulation={i}, stroke RR={strokeRR:<4.2f}, mi RR={miRR:<4.2f}")
    print(f"average of {nSimulations} simulations: stroke RR={np.mean(strokeRRList):<4.2f}, mi RR={np.mean(miRRList):4.2f}")
    print(f"sd of {nSimulations} simulations: stroke RR={np.std(strokeRRList):<4.2f}, mi RR={np.std(miRRList):<4.2f}")
